In [1]:
from pathlib import Path

from hloc import (
    extract_features,
    match_dense,
    match_features,
    pairs_from_retrieval,
    reconstruction,
    visualization,
)
from hloc.utils import viz_3d
import shutil
import time 


In [2]:

images = Path("takeout-1-001")

shutil.rmtree("outputs/sfm", ignore_errors=True)
outputs = Path("outputs/sfm/")
sfm_pairs = outputs / "pairs-netvlad.txt"
sfm_dir = outputs / "sfm_superpoint+lightglue"

retrieval_conf = extract_features.confs["netvlad"]
feature_conf = extract_features.confs["superpoint_aachen"]
matcher_conf = match_features.confs["superpoint+lightglue"]
# matcher_conf = match_dense.confs["loftr_aachen"]

time1 = time.time()

In [3]:
retrieval_path = extract_features.main(retrieval_conf, images, outputs)
pairs_from_retrieval.main(retrieval_path, sfm_pairs, num_matched=5)

[2025/06/26 18:19:37 hloc INFO] Extracting local features with configuration:
{'model': {'name': 'netvlad'},
 'output': 'global-feats-netvlad',
 'preprocessing': {'resize_max': 1024}}
[2025/06/26 18:19:37 hloc INFO] Found 27 images in root takeout-1-001.


  0%|          | 0/27 [00:00<?, ?it/s]/home/jourdelune/dev/colmap-api/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
  4%|▎         | 1/27 [00:05<02:29,  5.75s/it]


FileNotFoundError: [Errno 2] Unable to synchronously create file (unable to open file: name = 'outputs/sfm/global-feats-netvlad.h5', errno = 2, error message = 'No such file or directory', flags = 15, o_flags = c2)

In [ ]:
feature_path = extract_features.main(feature_conf, images, outputs)


[2025/06/26 16:12:49 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 3},
 'output': 'feats-superpoint-n4096-r1024',
 'preprocessing': {'grayscale': True, 'resize_max': 1024}}


[2025/06/26 16:12:49 hloc INFO] Found 27 images in root takeout-1-001.


Loaded SuperPoint model


100%|██████████| 27/27 [00:40<00:00,  1.48s/it]
[2025/06/26 16:13:30 hloc INFO] Finished exporting features.


In [ ]:
match_path = match_features.main(
    matcher_conf, sfm_pairs, feature_conf["output"], outputs
)

[2025/06/26 16:13:30 hloc INFO] Matching local features with configuration:
{'model': {'features': 'superpoint', 'name': 'lightglue'},
 'output': 'matches-superpoint-lightglue'}
/home/jourdelune/dev/colmap-api/.venv/lib/python3.10/site-packages/lightglue/lightglue.py:24: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
100%|██████████| 85/85 [01:00<00:00,  1.41it/s]
[2025/06/26 16:14:36 hloc INFO] Finished exporting matches.


In [ ]:
model = reconstruction.main(sfm_dir, images, sfm_pairs, feature_path, match_path)

[2025/06/26 16:14:36 hloc INFO] Creating an empty database...
[2025/06/26 16:14:36 hloc INFO] Importing images into the database...
[2025/06/26 16:14:43 hloc INFO] Importing features into the database...
100%|██████████| 27/27 [00:00<00:00, 704.39it/s]
[2025/06/26 16:14:44 hloc INFO] Importing matches into the database...
100%|██████████| 135/135 [00:00<00:00, 1493.29it/s]
[2025/06/26 16:14:44 hloc INFO] Performing geometric verification of the matches...
I20250626 16:14:44.144139 135373606819392 misc.cc:44] 
Feature matching
I20250626 16:14:44.148484 135373615212096 sift.cc:1432] Creating SIFT CPU feature matcher
I20250626 16:14:44.148489 135373631997504 sift.cc:1432] Creating SIFT CPU feature matcher
I20250626 16:14:44.148563 135373623604800 sift.cc:1432] Creating SIFT CPU feature matcher
I20250626 16:14:44.149259 135373697054272 sift.cc:1432] Creating SIFT CPU feature matcher
I20250626 16:14:44.149567 135373686289984 sift.cc:1432] Creating SIFT CPU feature matcher
I20250626 16:14:44

In [ ]:
time2 = time.time()
print(f"Total time taken: {time2 - time1} seconds")

Total time taken: 207.8719551563263 seconds


In [ ]:
fig = viz_3d.init_figure()
viz_3d.plot_reconstruction(fig, model, color='rgba(255,0,0,0.5)', name="mapping", points_rgb=True)
fig.show()

# Test the api

In [1]:
import os
import time

import requests


def post_images_to_api(images_dir, api_url="http://localhost:8000/sfm", multiply=1):
    files = []
    image_files = [
        f
        for f in os.listdir(images_dir)
        if os.path.isfile(os.path.join(images_dir, f))
        and f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]
    if not image_files:
        print("No images found in the directory.")
        return

    for i in range(multiply):
        for filename in image_files:
            filepath = os.path.join(images_dir, filename)
            # Ajoute un suffixe pour éviter les collisions de noms lors de la duplication
            send_name = (
                f"{os.path.splitext(filename)[0]}_copy{i}{os.path.splitext(filename)[1]}"
                if multiply > 1
                else filename
            )
            files.append(("images", (send_name, open(filepath, "rb"), "image/jpeg")))

    response = requests.post(api_url, files=files)
    print(f"Response status code: {response.status_code}")
    print(f"Response content: {response.text}")


time1 = time.time()
post_images_to_api(
    "takeout-1-001",
    api_url="http://localhost:8000/sfm",
    multiply=1,
)
print("API call completed.")
time2 = time.time()
print(f"API call time taken: {time2 - time1} seconds")

ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sfm (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x6ffdf65f5ea0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [3]:
import requests

# Prepare images
files = [
    ('images', open('/home/jourdelune/dev/colmap-api/takeout-1-001/PXL_20250619_090207768.jpg', 'rb')),
    ('images', open('/home/jourdelune/dev/colmap-api/takeout-1-001/PXL_20250619_090211404.jpg', 'rb')),
]

# Optional parameters
data = {
    'retrieval_conf_key': 'netvlad',
    'feature_conf_key': 'superpoint_aachen',
    'matcher_conf_key': 'superpoint+lightglue'
}

# Make request
response = requests.post('http://localhost:8000/sfm', files=files, data=data)

if response.status_code == 200:
    with open('reconstruction.zip', 'wb') as f:
        f.write(response.content)
    print("SfM reconstruction completed successfully!")
else:
    print(f"Error: {response.json()}")

Error: {'error': 'Internal server error: DataLoader worker (pid(s) 153) exited unexpectedly'}
